##### Copyright 2020 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Copyright 2020 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/hub/tutorials/tf2_object_detection"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/hub/tutorials/tf2_object_detection.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/hub/tutorials/tf2_object_detection.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/hub/tutorials/tf2_object_detection.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
  <td><a href="https://tfhub.dev/tensorflow/collections/object_detection/1"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">TF Hub 모델 보기</a></td>
</table>

# TensorFlow Hub 객체 감지 Colab

TensorFlow Hub 객체 감지 Colab에 오신 것을 환영합니다! 이 노트북에서는 이미지에서 "즉시 사용 가능한" 객체 감지 모델을 실행하는 단계를 안내합니다.

### 더 많은 모델

[이](https://tfhub.dev/tensorflow/collections/object_detection/1) 컬렉션에는 COCO 2017 데이터세트에서 훈련된 TF 2 객체 감지 모델이 포함되어 있습니다. 현재, [tfhub.dev](https://tfhub.dev/s?module-type=image-object-detection)에서 호스팅되는 모든 객체 감지 모델은 [여기](https://tfhub.dev/)에서 찾을 수 있습니다.

## 가져오기 및 설정

기본 가져오기부터 시작하겠습니다.

In [2]:
from google.colab import drive
drive.mount('/content/google_drive')

MessageError: ignored

In [ ]:
!nvidia-smi

Fri Nov 11 01:08:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# This Colab requires TF 2.5.
!pip install -U "tensorflow>=2.5"

In [ ]:
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import pathlib

import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen

import tensorflow as tf
# import tensorflow_hub as hub

tf.get_logger().setLevel('ERROR')

In [ ]:
import re
from shutil import copyfile
import argparse
import math
import random

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

## 유틸리티



## 시각화 도구

적절하게 감지된 상자, 키포인트 및 세분화로 이미지를 시각화하기 위해 TensorFlow Object Detection API를 사용합니다. 설치를 위해 리포지토리를 복제합니다.

In [ ]:
# Clone the tensorflow models repository
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3529, done.
remote: Counting objects: 100% (3529/3529), done.
remote: Compressing objects: 100% (2942/2942), done.
remote: Total 3529 (delta 931), reused 1511 (delta 533), pack-reused 0
Receiving objects: 100% (3529/3529), 47.00 MiB | 12.65 MiB/s, done.
Resolving deltas: 100% (931/931), done.


In [ ]:
# tar -zcvf [파일명.tar.gz] [폴더명]
!tar -zcvf tensorflow_odapi.tar.gz ./models/

In [ ]:
!cp /content/tensorflow_odapi.tar.gz "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/tensorflow_od_api"

Object Detection API 설치하기

Installation of the Object Detection API is achieved by installing the object_detection package. This is done by running the following commands from within Tensorflow\models\research:

In [ ]:
%%bash
sudo apt install -y protobuf-compiler
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .


## Tensorflow Object Detection API 설치 확인

In [ ]:
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

In [ ]:
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

%matplotlib inline

In [ ]:
!mkdir "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace"

In [ ]:
!mkdir "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/scripts"

In [ ]:
!mkdir "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/scripts/preprocessing"

In [ ]:
!mkdir "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo"

This folder will be used to store all *.csv files and the respective TensorFlow *.record files, which contain the list of annotations for our dataset images.

In [ ]:
!mkdir "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/annotations"

This folder will be used to store exported versions of our trained model(s).

In [ ]:
!mkdir "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/exported-models"

In [ ]:
!mkdir "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/images"

In [ ]:
!mkdir "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/images/test"

In [ ]:
!mkdir "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/images/train"

In [ ]:
!mkdir "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/models"

In [ ]:
!mkdir "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/pre-trained-models"

In [ ]:
# !touch ./workspace/training_demo/README.md

In [ ]:
# !pwd

In [ ]:
# !cp -r "/content/google_drive/MyDrive/datasets/VOCdevkit/VOC2012/" ./

In [ ]:
!cp "/content/google_drive/MyDrive/datasets/VOCtrainval_11-May-2012.tar" ./

In [ ]:
!tar -xvf ./VOCtrainval_11-May-2012.tar 

http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz \

In [ ]:
imageDir = "/content/VOCdevkit/VOC2012/JPEGImages"

In [ ]:
images = [f for f in os.listdir(imageDir)
              if re.search(r'([a-zA-Z0-9\s_\\.\-\(\):])+(?i)(.jpg|.jpeg|.png)$', f)]        # 리눅스 정규식 표현 : 파일에 a-z,A-Z,0-9라 적혀있는 파일이면 모두 가져온다.

In [ ]:
ratio=0.1

In [ ]:
num_images = len(images)
num_test_images = math.ceil(ratio*num_images)
print(num_test_images)

1713


In [ ]:
train_dir = "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/images/train"
test_dir = "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/images/test"

In [ ]:
source = imageDir

In [ ]:
source

'/content/VOCdevkit/VOC2012/JPEGImages'

In [ ]:
# source = source.replace('\\', '/')

In [ ]:
copy_xml=True

In [ ]:
sourceXML="/content/VOCdevkit/VOC2012/Annotations"

In [ ]:
for i in range(num_test_images):
  idx = random.randint(0, len(images)-1)
  filename = images[idx]
  copyfile(os.path.join(source, filename), os.path.join(test_dir, filename))
  if copy_xml:
    xml_filename = os.path.splitext(filename)[0]+'.xml'
    copyfile(os.path.join(sourceXML, xml_filename), os.path.join(test_dir,xml_filename))

  images.remove(images[idx])

for filename in images:
  copyfile(os.path.join(source, filename), os.path.join(train_dir, filename))
  if copy_xml:
    xml_filename = os.path.splitext(filename)[0]+'.xml'
    copyfile(os.path.join(sourceXML, xml_filename), os.path.join(train_dir, xml_filename))

* `%cd`: 지속적인 디렉토리 변경 </br>
  %cd sample_data/

In [ ]:
%cd "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/images/test"

/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/images/test


*.jpeg + *.xml = ?

In [ ]:
!ls -l | grep ^- | wc -l

3426


In [1]:
%cd "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/images/train"

[Errno 2] No such file or directory: '/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/images/train'
/content


In [ ]:
!ls -l | grep ^- | wc -l

In [ ]:
!pwd

/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/images/test


In [ ]:
# !cd ./

In [ ]:
# !pwd

/content/workspace/training_demo/images/test


In [ ]:
%cd /content

/content


In [ ]:
!pwd

/content


/content/models/research/object_detection/data/pascal_label_map.pbtxt

위 파일을 복사해서 파일 이름은 label_map.pbtxt로 변경한 후</br>
/content/workspace/training_demo/annotations에 저장한다

In [ ]:
!cp "/content/models/research/object_detection/data/pascal_label_map.pbtxt" \
"/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/annotations"

In [ ]:
!mv "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/annotations/pascal_label_map.pbtxt" \
"/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/annotations/label_map.pbtxt"

**PATH_TO_IMAGES_FOLDER**=*C:/Users/sglvladi/Documents/Tensorflow/workspace/training_demo/images*</br>

For example</br>

python partition_dataset.py -x -i C:/Users/sglvladi/Documents/Tensorflow/workspace/training_demo/images -r 0.1

In [ ]:
PATH_TO_IMAGES_FOLDER="/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/images"
PATH_TO_ANNOTATIONS_FOLDER="/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/annotations"

## generate_tfrecord.py을 아래 경로에 복사
/content/scripts/preprocessing/generate_tfrecord.py

In [ ]:
!cp "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/generate_tfrecord.py" \
"/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/scripts/preprocessing"

In [ ]:
%cd "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/scripts/preprocessing"

/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/scripts/preprocessing


In [ ]:
!pwd

/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/scripts/preprocessing


In [ ]:
!ls -al

total 7
-rw------- 1 root root 6962 Nov 11 01:46 generate_tfrecord.py


In [ ]:
# Create train data:
!python generate_tfrecord.py \
-x "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/images/train" \
-l "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/annotations/label_map.pbtxt" \
-o "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/annotations/train.record"

2022-11-11 01:49:09.366824: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Successfully created the TFRecord file: /content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/annotations/train.record


In [ ]:
# Create test data:
# !python /content/scripts/preprocessing/generate_tfrecord.py \
# -x /content/workspace/training_demo/images/test \
# -l /content/workspace/training_demo/annotations/label_map.pbtxt \
# -o /content/workspace/training_demo/annotations/test.record

!python generate_tfrecord.py \
-x "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/images/test" \
-l "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/annotations/label_map.pbtxt" \
-o "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/annotations/test.record"

2022-11-11 01:50:42.440457: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Successfully created the TFRecord file: /content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/annotations/test.record


In [ ]:
!pwd

/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/scripts/preprocessing


In [ ]:
%cd /content

/content


In [ ]:
!pwd

/content


In [ ]:
!mkdir ./pre-trained-models

In [ ]:
!tar xvfzp "/content/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz" \
-C "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/pre-trained-models"

faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/
faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/
faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/checkpoint
faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0.index
faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/pipeline.config
faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/saved_model/
faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/saved_model/saved_model.pb
faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/saved_model/variables/
faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/saved_model/variables/variables.index


In [ ]:
!mkdir "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/models/my_faster_rcnn_resnet50_v1"

## pipeline.confg
Tensorflow object detection Model Zoo에서 다운로드한 Pre-trained Model의 디렉토리에 있는 pipeline.config을 수정하여 Transfer Learning을 수행한다.

In [ ]:
# !cp "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/pipeline.config" /content/workspace/training_demo/models/my_ssd_resnet50_v1_fpn

In [ ]:
!cp "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/pipeline.config" \
"/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/models/my_faster_rcnn_resnet50_v1"

In [ ]:
run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)

In [ ]:
# !cd /content/workspace/training_demo

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir="/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/models/my_faster_rcnn_resnet50_v1"

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
--model_dir="/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/models/my_faster_rcnn_resnet50_v1" \
--pipeline_config_path="/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/models/my_faster_rcnn_resnet50_v1/pipeline.config"

## save model!!!

In [ ]:
# !cp /content/models/research/object_detection/exporter_main_v2.py /content/workspace/training_demo

In [ ]:
!mkdir "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/exported-models/my_faster_rcnn_resnet50_v1_1"

In [ ]:
PATH_TO_MODEL_DIR = "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/exported-models/my_faster_rcnn_resnet50_v1_1"
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"
print(PATH_TO_SAVED_MODEL)

/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/exported-models/my_faster_rcnn_resnet50_v1_1/saved_model


In [ ]:
!python /content/models/research/object_detection/exporter_main_v2.py \
--input_type image_tensor \
--pipeline_config_path "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/models/my_faster_rcnn_resnet50_v1/pipeline.config" \
--trained_checkpoint_dir "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/models/my_faster_rcnn_resnet50_v1" \
--output_directory "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/exported-models/my_faster_rcnn_resnet50_v1_1"

In [ ]:
%tensorboard --logdir="/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/models/my_faster_rcnn_resnet50_v1"

In [ ]:
# %tensorboard --logdir="/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/exported-models/my_faster_rcnn_resnet50_v1_1"

In [ ]:
!pwd

/content/scripts/preprocessing


In [ ]:
# !tar -cvf ./faster_rcnn_ob_model.tar "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/exported-models/my_faster_rcnn_resnet50_v1_1"

In [ ]:
from tensorflow.core.util import event_pb2
from pathlib import Path

In [ ]:
train_metrics = [
    "Loss/BoxClassifierLoss/localization_loss",
    "Loss/RPNLoss/localization_loss",
    "Loss/RPNLoss/objectness_loss",
    "Loss/regularization_loss",
    "Loss/total_loss",
    "learning_rate",
]

In [ ]:
event_files = [str(f) for f in Path('/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/models/my_faster_rcnn_resnet50_v1/train').rglob('events.out.*')]

In [ ]:
print(event_files)

['/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/models/my_faster_rcnn_resnet50_v1/train/events.out.tfevents.1668133456.d9f17c322636.2819.0.v2', '/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/models/my_faster_rcnn_resnet50_v1/train/events.out.tfevents.1668133586.d9f17c322636.2928.0.v2']


In [ ]:
len(event_files)

2

In [ ]:
# for event_file in event_files:
#     serialized_examples = tf.data.TFRecordDataset(event_file)
#     # print(len(serialized_examples))
#     print(type(serialized_examples))
#     event = event_pb2.Event.FromString(serialized_examples.numpy())
#     for value in event.summary.value:
#       if value.tag in train_metrics:
#         t = tf.make_ndarray(value.tensor)
#         print(value.tag, event.step, t, type(t))
#     # for serialized_example in serialized_examples:
#     #     event = event_pb2.Event.FromString(serialized_example.numpy())
#     #     for value in event.summary.value:
#     #         if value.tag in train_metrics:
#     #           t = tf.make_ndarray(value.tensor)
#     #           print(value.tag, event.step, t, type(t))

In [ ]:
print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...Done! Took 7.959916591644287 seconds


In [ ]:
PATH_TO_LABELS = '/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/annotations/label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [ ]:
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))

In [ ]:
TEST_IMAGE_PATH = "/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/images/test"

In [ ]:
IMAGE_PATHS = [f for f in os.listdir('/content/google_drive/MyDrive/Colab Notebooks/pre_trained model_train/faster_rcnn_50_v1/workspace/training_demo/images/test')
              if re.search(r'([a-zA-Z0-9\s_\\.\-\(\):])+(?i)(.jpg|.jpeg)$', f)] 

In [ ]:
print(IMAGE_PATHS)

['2010_000655.jpg', '2008_003621.jpg', '2009_003627.jpg', '2011_003495.jpg', '2009_002452.jpg', '2007_003330.jpg', '2011_004589.jpg', '2010_003861.jpg', '2011_004197.jpg', '2011_007087.jpg', '2007_009052.jpg', '2011_002947.jpg', '2009_000512.jpg', '2011_001476.jpg', '2011_003436.jpg', '2010_004575.jpg', '2008_005117.jpg', '2009_004164.jpg', '2011_005286.jpg', '2007_002462.jpg', '2009_000419.jpg', '2009_004545.jpg', '2010_005500.jpg', '2009_003115.jpg', '2011_005258.jpg', '2012_001673.jpg', '2009_001252.jpg', '2010_003326.jpg', '2008_006433.jpg', '2012_000516.jpg', '2009_002066.jpg', '2008_007476.jpg', '2008_002167.jpg', '2008_003989.jpg', '2008_002850.jpg', '2007_001420.jpg', '2011_001330.jpg', '2012_002905.jpg', '2011_003836.jpg', '2009_004092.jpg', '2008_003622.jpg', '2008_008388.jpg', '2011_000928.jpg', '2010_004743.jpg', '2008_008221.jpg', '2009_005083.jpg', '2012_000554.jpg', '2011_000418.jpg', '2012_002355.jpg', '2012_001518.jpg', '2008_008262.jpg', '2009_004002.jpg', '2008_00407

In [ ]:
len(IMAGE_PATHS)

1713

In [ ]:
for image_path in IMAGE_PATHS:

    print('Running inference for {}... '.format(image_path), end='')

    image_path = TEST_IMAGE_PATH + '/' + image_path
    image_np = load_image_into_numpy_array(image_path)
    
    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.30,
          agnostic_mode=False)

    plt.figure(figsize=(24,24))
    plt.imshow(image_np_with_detections)
    print('Done')

plt.show()


이제 나중에 필요한 종속성을 가져올 수 있습니다.

In [ ]:
label_id_offset = 0
image_np_with_detections = image_np.copy()

# Use keypoints if available in detections
keypoints, keypoint_scores = None, None
if 'detection_keypoints' in result:
  keypoints = result['detection_keypoints'][0]
  keypoint_scores = result['detection_keypoint_scores'][0]

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections[0],
      result['detection_boxes'][0],
      (result['detection_classes'][0] + label_id_offset).astype(int),
      result['detection_scores'][0],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.30,
      agnostic_mode=False,
      keypoints=keypoints,
      keypoint_scores=keypoint_scores,
      keypoint_edges=COCO17_HUMAN_POSE_KEYPOINTS)

plt.figure(figsize=(24,32))
plt.imshow(image_np_with_detections[0])
plt.show()

## [옵션]

사용 가능한 객체 감지 모델 중에는 Mask R-CNN이 있으며, 이 모델의 출력은 인스턴스 분할을 허용합니다.

이를 시각화하기 위해 이전에 했던 것과 같은 방법을 사용하지만 다른 매개변수를 추가합니다. `instance_masks=output_dict.get('detection_masks_reframed', None)`


In [ ]:
# Handle models with masks:
image_np_with_mask = image_np.copy()

if 'detection_masks' in result:
  # we need to convert np.arrays to tensors
  detection_masks = tf.convert_to_tensor(result['detection_masks'][0])
  detection_boxes = tf.convert_to_tensor(result['detection_boxes'][0])

  # Reframe the bbox mask to the image size.
  detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes,
              image_np.shape[1], image_np.shape[2])
  detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                      tf.uint8)
  result['detection_masks_reframed'] = detection_masks_reframed.numpy()

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_mask[0],
      result['detection_boxes'][0],
      (result['detection_classes'][0] + label_id_offset).astype(int),
      result['detection_scores'][0],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.30,
      agnostic_mode=False,
      instance_masks=result.get('detection_masks_reframed', None),
      line_thickness=8)

plt.figure(figsize=(24,32))
plt.imshow(image_np_with_mask[0])
plt.show()